In [1]:
pip install d2spy leafmap

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datetime import date

from d2spy.workspace import Workspace

import os

import leafmap

In [2]:
# Connect to Purdue hosted D2S instance. It will prompt for your D2S password.
workspace = Workspace.connect("https://ps2.d2s.org", "jgillan@arizona.edu")

In [3]:
# Get list of all your projects
projects = workspace.get_projects()

# Check if there are any projects
if len(projects) > 0:
    # Loop through all projects and print each one
    for project in projects:
        print(project)
else:
    print("Please create a project before proceeding with this guide.")


Project(title='Holiday Beach', description='After Hurricane Harvey Survey')
Project(title='Purdue PUSH', description='Purdue University Student Health Service building')
Project(title='OsaLarge', description='Large area including the Osa Conservation base camp.')
Project(title='Biosphere 2', description='UAS Summit On Open Science & Data Management')
Project(title='Santa Rita Experimental Range', description='Drone Imagery from 2019 collection')
Project(title='Cornell Wheat - 2023', description='T3 <-> UASHUB')
Project(title='Peanut Demo', description='Demonstrating cloud data processing example')
Project(title='Paradise Valley Country Club', description='Topographic surveying of the golf course for the purpose of installing subsurface drip irrigation. ')
Project(title='Santa Cruz River - Pima County Flood Control', description='Contract work to image a stretch of the Santa Cruz River in Tucson, Az for purposes of mapping vegetation. ')
Project(title='Swetnam House', description='Drone

In [4]:
# Get list of all flights for a project
flights = projects[4].get_flights()

# Check if there are any flights
if len(flights) > 0:
    # Loop through all flights and print each one
    for flight in flights:
        print(flight)
else:
    print("No flights found for this project.")


Flight(acquisition_date='2019-05-08', name=None, altitude=40.0, side_overlap=70.0, forward_overlap=80.0, sensor='RGB', platform='Phantom_4')
Flight(acquisition_date='2019-05-20', name=None, altitude=40.0, side_overlap=70.0, forward_overlap=80.0, sensor='RGB', platform='Phantom_4')


In [5]:
# Get list of data products from a flight
data_products = flights[0].get_data_products()

# Check if there are any data products
if len(data_products) > 0:
    # Loop through all data products and print their URLs
    for product in data_products:
        print(product.url)
else:
    print("No data products found for this flight.")


https://ps2.d2s.org/static/projects/2f939aa4-95e0-4b33-b0bb-e144207044ab/flights/9fce9f5c-d2eb-47b4-b7eb-eee513c5686f/data_products/5aa35c1f-c3c7-4316-a841-9ae2afcb8ae2/48d5a92a-a85f-44bf-b7e3-d4c217013e6f.tif
https://ps2.d2s.org/static/projects/2f939aa4-95e0-4b33-b0bb-e144207044ab/flights/9fce9f5c-d2eb-47b4-b7eb-eee513c5686f/data_products/4e66c08f-18d6-48a8-b1f7-78c8b07897ea/96b0485d-322e-4484-994d-3dcf6648dc39.tif


In [6]:
os.environ["TITILER_ENDPOINT"] = "https://tt.d2s.org"

In [7]:
# Interactive leafmap Map
m = leafmap.Map()

# URL for a D2S hosted GeoTIFF data product
ortho_url = "https://ps2.d2s.org/static/projects/2f939aa4-95e0-4b33-b0bb-e144207044ab/flights/9fce9f5c-d2eb-47b4-b7eb-eee513c5686f/data_products/5aa35c1f-c3c7-4316-a841-9ae2afcb8ae2/48d5a92a-a85f-44bf-b7e3-d4c217013e6f.tif"

# Add a publicly available data product to the map
m.add_cog_layer(ortho_url, name="Orthomosaic")

# If you want to display a private data product, comment out the previously line and uncomment the below m.add_cog_layer line
# Add a private data product to the map
# m.add_cog_layer(f"{ortho_url}?API_KEY={api_key}", name="DSM", colormap_name="rainbow")

# Display the map
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
# Removes access token from future requests
workspace.logout()